In [55]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
import re

In [56]:

def make_lag(data,lags,week):
    for lag in range(1,lags+1):
            if lag==0 and week==0:
                  continue
            data['lag_{}_{}'.format(lag,week)]=data.groupby('Store')['Sales'].shift(lag+7*week)
    return data
def clean_col_name(name):
    # 将 ( ) , = 以及空格替换为 _
    return re.sub(r'[(),=\s-]', '_', str(name))
def rmspe(y_true, y_pred):
    mask = y_true > 0
    
    # 2. 如果验证集中全是 0 (极少见，但在某些极小的切分下可能发生)，直接返回 0
    if np.sum(mask) == 0:
        return 0.0
    
    # 3. 只取大于 0 的部分进行计算
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]
    
    # 4. 标准 RMSPE 公式
    score = np.sqrt(np.mean(((y_true_clean - y_pred_clean) / y_true_clean) ** 2))
    
    return score


In [57]:
def beseline_engineering(data):
    data['Date']=pd.to_datetime(data['Date'])
    data.sort_values(by=['Store','Date'],inplace=True)
    data.set_index('Date',inplace=True)
    temp=pd.DataFrame()
    temp=data[['Store','Sales']].copy()
    print(temp.columns)
    make_lag(temp,3,0)
    make_lag(temp,1,1)
    temp['rolling_mean_7'] = (
    temp
        .groupby('Store')['Sales']
        .rolling(7, min_periods=1)
        .mean()
        .shift(1)
        .reset_index(level=0, drop=True)
    )
    temp['rolling_std_7'] = (
    temp
        .groupby('Store')['Sales']
        .rolling(7, min_periods=1)
        .std()
        .shift(1)
        .fillna(0)
        .reset_index(level=0, drop=True)
    )
    temp[temp['Store']==3].head(10)
    baseline_features=data.copy()
    baseline_features.drop(['Customers','CompetitionDistance',	'CompetitionOpenSinceMonth',
                        	'CompetitionOpenSinceYear',	'Promo2','Promo2SinceWeek','Promo2SinceYear','PromoInterval'],axis=1,inplace=True)
    baseline_features['day']=((data.index - data.index.min())).days
    to_category=['Open','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment']
    baseline_features[to_category]=baseline_features[to_category].astype('category')
    baseline_features['DayofMonth']=data.index.day
    baseline_features.head(10)
    res=[]
    for Store,df_store in data.groupby('Store'):
        fouriers = CalendarFourier(freq="A", order=10)#生成日历傅里叶特征
        dp = DeterministicProcess(
            index=df_store.index,
            constant=True,               #是否包含常数项
            order=1,                     #线性趋势
            seasonal=False,               #是否包含季节性虚拟变量
            additional_terms=[fouriers], #添加傅里叶特征
            drop=True,           
        )
        X=dp.in_sample()  #生成样本内的确定性特征
        X['Store']=Store
        X['Date']=df_store.index
        res.append(X)
    X_dp = pd.concat(res, ignore_index=True)
    baseline_features=baseline_features.merge(X_dp,on=['Store','Date'],how='left')
    y = data["Sales"]               #目标变量
    baseline_features.head(10)
    baseline_features=baseline_features.merge(right=temp,on=['Date','Store'],how='left')
    baseline_features.dropna(inplace=True)
    baseline_features=baseline_features[baseline_features['Date']>'2013-01-08']
    baseline_features=baseline_features[baseline_features['Open']==1]
    baseline_features.sort_values(by='Date',inplace=True)
    # 2. 重命名 DataFrame 的所有列
    baseline_features.columns = [clean_col_name(c) for c in baseline_features.columns]

    # 3. 非常重要：因为列名变了，必须重新生成 feature_cols 列表！
    exclude_cols = ['Index', 'Date', 'Sales', 'log_Sales', 'Open']
    # 确保排除列也用了清洗过的名字（如果排除列里原本没有特殊字符，其实不需要，但为了保险）
    exclude_cols = [clean_col_name(c) for c in exclude_cols]

    feature_cols = [c for c in baseline_features.columns if c not in exclude_cols]

    return baseline_features,y

In [58]:
main=pd.read_csv('../data/train.csv')
sub=pd.read_csv('../data/store.csv')
data=pd.merge(main,sub,on='Store',how='left')
data['Date']=pd.to_datetime(data['Date'])
data.sort_values(by=['Store','Date'],inplace=True)
tmp=data.copy()
#data.set_index('Date',inplace=True)

/tmp/ipykernel_1800/1250609072.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  main=pd.read_csv('../data/train.csv')


In [59]:
data.index

Index([1016095, 1014980, 1013865, 1012750, 1011635, 1010520, 1009405, 1008290,
       1007175, 1006060,
       ...
         11149,   10034,    8919,    7804,    6689,    5574,    4459,    3344,
          2229,    1114],
      dtype='int64', length=1017209)

In [60]:
feature,y=beseline_engineering(data)
feature.head()

Index(['Store', 'Sales'], dtype='object')


/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)


,Store,Date,DayOfWeek,Sales_x,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,cos_9_freq_YE_DEC_,sin_10_freq_YE_DEC_,cos_10_freq_YE_DEC_,Sales_y,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
8,1,2013-01-09,3,5471,1,1,0,1,c,a,...,0.325342,0.981306,0.192452,5471,5580.0,7176.0,0.0,0.0,4585.142857,2231.018633
115962,128,2013-01-09,3,5572,1,1,0,1,d,c,...,0.325342,0.981306,0.192452,5572,6465.0,7819.0,0.0,0.0,4945.428571,2475.318488
783816,861,2013-01-09,3,1559,1,1,0,0,c,c,...,0.325342,0.981306,0.192452,1559,1929.0,1711.0,0.0,0.0,1614.857143,745.003899
784758,862,2013-01-09,3,8640,1,1,0,0,a,c,...,0.325342,0.981306,0.192452,8640,10733.0,12571.0,0.0,0.0,7751.428571,4003.608949
115204,127,2013-01-09,3,6133,1,1,0,0,d,a,...,0.325342,0.981306,0.192452,6133,5431.0,7204.0,0.0,0.0,3892.714286,2273.777013


In [61]:
feature.columns

Index(['Store', 'Date', 'DayOfWeek', 'Sales_x', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'day',
       'DayofMonth', 'const', 'trend', 'sin_1_freq_YE_DEC_',
       'cos_1_freq_YE_DEC_', 'sin_2_freq_YE_DEC_', 'cos_2_freq_YE_DEC_',
       'sin_3_freq_YE_DEC_', 'cos_3_freq_YE_DEC_', 'sin_4_freq_YE_DEC_',
       'cos_4_freq_YE_DEC_', 'sin_5_freq_YE_DEC_', 'cos_5_freq_YE_DEC_',
       'sin_6_freq_YE_DEC_', 'cos_6_freq_YE_DEC_', 'sin_7_freq_YE_DEC_',
       'cos_7_freq_YE_DEC_', 'sin_8_freq_YE_DEC_', 'cos_8_freq_YE_DEC_',
       'sin_9_freq_YE_DEC_', 'cos_9_freq_YE_DEC_', 'sin_10_freq_YE_DEC_',
       'cos_10_freq_YE_DEC_', 'Sales_y', 'lag_1_0', 'lag_2_0', 'lag_3_0',
       'lag_1_1', 'rolling_mean_7', 'rolling_std_7'],
      dtype='object')

In [62]:
data.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [64]:
#加入更多周周期滞后信息
nf=pd.DataFrame()
nf[['Store','Date']]=tmp[['Store','Date']].copy()
nf['lag_14']=tmp.groupby('Store')['Sales'].shift(14).copy()
nf['lag_21']=tmp.groupby('Store')['Sales'].shift(21).copy()
nf['lag_28']=tmp.groupby('Store')['Sales'].shift(28).copy()
nf['rolling_mean_28']=tmp.groupby('Store')['Sales'].transform(lambda x: x.shift(1).rolling(28,center=False,min_periods=1).mean()).copy()
nf['rolling_std_28']=tmp.groupby('Store')['Sales'].transform(lambda x: x.shift(1).rolling(28,center=False,min_periods=1).std()).copy()
nf[['lag_14','lag_21','lag_28','rolling_mean_28','rolling_std_28']].head()
nf.dropna(inplace=True)
nf=nf.merge(feature,on=['Store','Date'],how='inner')
nf.head()

,Store,Date,lag_14,lag_21,lag_28,rolling_mean_28,rolling_std_28,DayOfWeek,Sales_x,Open,...,cos_9_freq_YE_DEC_,sin_10_freq_YE_DEC_,cos_10_freq_YE_DEC_,Sales_y,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
0,1,2013-01-29,3900.0,5580.0,0.0,4121.285714,2075.631146,2,3725,1,...,-0.365723,-0.994218,0.107381,3725,4055.0,0.0,5598.0,5394.0,4533.142857,2080.017823
1,1,2013-01-30,4008.0,5471.0,5530.0,4254.321429,1914.845455,3,4601,1,...,-0.217723,-0.961130,0.276097,4601,3725.0,4055.0,0.0,5720.0,4248.142857,2026.274696
2,1,2013-01-31,4044.0,4892.0,4327.0,4221.142857,1899.913267,4,4709,1,...,-0.064508,-0.899631,0.436651,4709,4601.0,3725.0,4055.0,5578.0,4108.571429,1951.681400
3,1,2013-02-01,4127.0,4881.0,4486.0,4234.785714,1902.071860,5,5633,1,...,0.090252,-0.811539,0.584298,5633,4709.0,4601.0,3725.0,5195.0,4039.142857,1914.889329
4,1,2013-02-02,5182.0,4952.0,4997.0,4275.750000,1919.949818,6,5970,1,...,0.242850,-0.699458,0.714673,5970,5633.0,4709.0,4601.0,5586.0,4045.857143,1921.288841


In [65]:
best_parms={
            'objective': 'regression',       # 回归问题
            'metric': 'rmse',                # 训练时优化 RMSE (对应 log_sales)
            'boosting_type': 'gbdt',
            'verbosity': -1,                 # 不打印乱七八糟的日志
            'n_jobs': -1,                    # 使用所有 CPU 核心
            'seed': 42,
            'learning_rate': 0.05801820833937896,
            'num_leaves': 144, 
            'max_depth': 13, 
            'min_child_samples': 18,
            'subsample': 0.9129847936149911,
            'colsample_bytree': 0.9384389823955829,
            'reg_alpha': 0.6588925840200691,
            'reg_lambda': 1.5944281549844126}

In [66]:
from sklearn.model_selection import TimeSeriesSplit
def train_lgbm(feature,y,params,k):
    tscv = TimeSeriesSplit(n_splits=k)
    rmspe_list=[]
    for fold_n,(train_index, val_index )in tscv.split(feature):
        X_train, X_val = feature.iloc[train_index], feature.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=1000,
                        valid_sets=[lgb_train, lgb_eval],
                        early_stopping_rounds=50,
                        verbose_eval=False)
        y_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
        score = rmspe(y_val.values, y_pred)
        rmspe_list.append(score)
        print(f"Fold {fold_n+1} RMSPE: {score:.5f}")
    print(f"\n基线模型平均 RMSPE: {np.mean(rmspe_list):.5f}")
    print(f"基线模型标准差: {np.std(rmspe_list):.5f}")

In [72]:
def train_lgbm_by_date(feature, y, params, k, date_col='day'):
    """
    基于整型日期列 (int) 的时间序列交叉验证
    
    Parameters
    ----------
    feature : pd.DataFrame
        特征矩阵，必须包含 date_col
    y : pd.Series
        目标变量 (注意：这里必须传入 log_Sales)
    params : dict
        LightGBM 参数
    k : int
        折数 (Folds)
    date_col : str
        日期列名，该列必须是 int 类型 (e.g., 1, 2, 3...)
    """

    # 1. 取出所有唯一日期（整型），并排序
    unique_dates = np.sort(feature[date_col].unique())
    rmspe_list = []
    
    # 2. 构造 fold (按时间切分)
    # array_split 会尽量均分，保证每一折的时间跨度大致相同
    date_folds = np.array_split(unique_dates, k + 1)
    
    print(f"开始时间序列交叉验证 (Total samples: {len(feature)})...")
    
    for fold_n in range(k):
        # 3. 定义日期区间
        # 训练集：从第0块到第n块
        # 验证集：第 n+1 块
        train_dates = np.concatenate(date_folds[:fold_n + 1])
        val_dates   = date_folds[fold_n + 1]
        
        # 4. 筛选数据
        train_mask = feature[date_col].isin(train_dates)
        val_mask   = feature[date_col].isin(val_dates)
        
        # 剔除日期列，避免模型直接把 "Day" 当作特征死记硬背
        # (虽然 Day 本身作为 int 有时也可以是特征，但通常作为 split 依据更好)
        X_train = feature.loc[train_mask].drop(columns=[date_col])
        X_val   = feature.loc[val_mask].drop(columns=[date_col])
        
        y_train = y.loc[train_mask]
        y_val   = y.loc[val_mask] # 这里的 y_val 是 log 后的值
        
        # 5. 构建 Dataset
        # 显式告诉 LGBM 自动识别 category 列
        lgb_train = lgb.Dataset(X_train, y_train, categorical_feature='auto')
        lgb_eval  = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature='auto')
        
        # 6. 训练
        gbm = lgb.train(
            params,
            lgb_train,
            num_boost_round=2000, 
            valid_sets=[lgb_train, lgb_eval],
            valid_names=['train', 'val'],
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(period=0) # 静默模式
            ]
        )
        
        # 7. 预测与评估 (关键还原步骤)
        # 预测出 Log Sales
        y_pred_log = gbm.predict(X_val, num_iteration=gbm.best_iteration)
        
        # 还原为 Real Sales
        y_true_real = np.expm1(y_val.values)
        y_pred_real = np.expm1(y_pred_log)
        
        # 计算 RMSPE (必须用还原后的值计算)
        score = rmspe(y_true_real, y_pred_real)
        rmspe_list.append(score)
        
        # 打印信息 (使用 int 日期显示范围)
        print(f"Fold {fold_n + 1} | RMSPE: {score:.5f}")
        print(f"  Train Days: {train_dates.min()} -> {train_dates.max()}")
        print(f"  Val   Days: {val_dates.min()} -> {val_dates.max()}")
        print("-" * 40)

    # 8. 汇总结果
    print(f"\nFinal CV RMSPE: {np.mean(rmspe_list):.5f} ± {np.std(rmspe_list):.5f}")
    
    return rmspe_list

In [68]:
ny=np.log1p(nf['Sales_x'])
nX=nf.drop(['Date','Sales_x','Sales_y'],axis=1)
nX.head()

,Store,lag_14,lag_21,lag_28,rolling_mean_28,rolling_std_28,DayOfWeek,Open,Promo,StateHoliday,...,sin_9_freq_YE_DEC_,cos_9_freq_YE_DEC_,sin_10_freq_YE_DEC_,cos_10_freq_YE_DEC_,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
0,1,3900.0,5580.0,0.0,4121.285714,2075.631146,2,1,0,0,...,-0.930724,-0.365723,-0.994218,0.107381,4055.0,0.0,5598.0,5394.0,4533.142857,2080.017823
1,1,4008.0,5471.0,5530.0,4254.321429,1914.845455,3,1,0,0,...,-0.976011,-0.217723,-0.961130,0.276097,3725.0,4055.0,0.0,5720.0,4248.142857,2026.274696
2,1,4044.0,4892.0,4327.0,4221.142857,1899.913267,4,1,0,0,...,-0.997917,-0.064508,-0.899631,0.436651,4601.0,3725.0,4055.0,5578.0,4108.571429,1951.681400
3,1,4127.0,4881.0,4486.0,4234.785714,1902.071860,5,1,0,0,...,-0.995919,0.090252,-0.811539,0.584298,4709.0,4601.0,3725.0,5195.0,4039.142857,1914.889329
4,1,5182.0,4952.0,4997.0,4275.750000,1919.949818,6,1,0,0,...,-0.970064,0.242850,-0.699458,0.714673,5633.0,4709.0,4601.0,5586.0,4045.857143,1921.288841


In [69]:
nX[['Store','day']].head()

,Store,day
0,1,28
1,1,29
2,1,30
3,1,31
4,1,32


In [73]:
train_lgbm_by_date(nX, ny,best_parms,5, date_col='day')

开始时间序列交叉验证 (Total samples: 818845)...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[185]	train's rmse: 0.112136	val's rmse: 0.139387
Fold 1 | RMSPE: 0.14141
  Train Days: 28 -> 180
  Val   Days: 181 -> 333
----------------------------------------
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[43]	train's rmse: 0.131009	val's rmse: 0.214104
Fold 2 | RMSPE: 0.23693
  Train Days: 28 -> 333
  Val   Days: 334 -> 485
----------------------------------------
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[107]	train's rmse: 0.114198	val's rmse: 0.164329
Fold 3 | RMSPE: 0.15039
  Train Days: 28 -> 485
  Val   Days: 486 -> 637
----------------------------------------
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[462]	train's rmse: 0.0953996	val's rmse: 0.141698
Fold 4 | RMSPE: 0.34597
  Train Days

[np.float64(0.14141102811077833),
 np.float64(0.23692501732784577),
 np.float64(0.15039439054779222),
 np.float64(0.34596851972669723),
 np.float64(0.12258976498351959)]